## 导入数据

In [1]:
from load_data import *
import tensorflow as tf
import os


os.environ['CUDA_VISIBLE_DEVICES'] = '1' #使用 GPU 0
def use_gpu_polite(using_rate=0.6):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = using_rate
    return config

In [2]:
# 导入数据，train data 是一个字典， ebd embdding 是词向量矩阵，作为embedding层的初始参数
train_data, test_data, ebd_weights = load_imdb_data()
print(train_data.keys())

Load pickle data from ../data/imdb_data_3col.pkl
Original 134957 words in vocabulary.
After truncated low frequent word:
words num: 40000/134957; words freq: 0.981
Words exit in w2v file: 39210/40004, rate: 98.015198%
Shape of weight matrix: (40006, 50)
Train data shape: (25000, 500) label length: 25000
Test data shape: (25000, 500) label length: 25000
dict_keys(['data', 'data_len', 'label'])


## TextCNN模型，使用layer借口实现

In [3]:
class TextCNN_layer():
    def __init__(self, seq_length, class_num, ebd_weights, filter_num, filter_sizes = [2, 3, 4]):
        seed_num = 7

        self.seq_input = tf.placeholder(dtype=tf.int32, shape=[None, seq_length],
                                        name='sequence_input')
        self.sparse_label_input = tf.placeholder(dtype=tf.int32, shape=[None],
                                                 name='sparse_label')
        self.global_step = tf.Variable(0, trainable=False)
        self.global_step_op = tf.assign(self.global_step, self.global_step+1)

        embedding_dim = ebd_weights.shape[1]
        with tf.name_scope('embedding'):
            self.W = tf.Variable(initial_value=ebd_weights, name='W')
            self.embedding_layer = tf.nn.embedding_lookup(self.W, self.seq_input)
            # batch * seq_len * emb_dim * in_channel(1)
            self.embedding_layer_expand = tf.expand_dims(self.embedding_layer, axis=-1)

        pool_layers = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope('conv-maxpool-%s' % i):
                conv = tf.layers.conv2d(self.embedding_layer_expand, filter_num, [filter_size, embedding_dim],
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.1, seed=seed_num),
                                bias_initializer=tf.constant_initializer(0.1),
                                activation='relu', padding='VALID')
                pool = tf.layers.max_pooling2d(conv, pool_size=[seq_length-filter_size+1, 1], 
                                               strides=[1, 1], padding='VALID')
                pool_layers.append(pool)
        
        all_dim = len(filter_sizes) * filter_num
        self.pool_flatten = tf.reshape(tf.concat(pool_layers, -1), shape=[-1, all_dim])
        
#         with tf.name_scope('output'):
#             self.logits = tf.layers.dense(self.pool_flatten, class_num, 
#                                           kernel_initializer=tf.truncated_normal_initializer(seed=seed_num), 
#                                           bias_initializer=tf.constant_initializer(0.1))
#             self.prediction = tf.argmax(self.logits, axis=-1, output_type=tf.int32, name='prediction')

        with tf.name_scope('output'):
            W = tf.Variable(tf.truncated_normal([all_dim, class_num], seed=seed_num, name='W'))
            b = tf.Variable(tf.constant(0.1, shape=[class_num]), name='b')
            self.logits = tf.nn.xw_plus_b(self.pool_flatten, W, b, name='logits')
            self.prediction = tf.argmax(self.logits, axis=-1, output_type=tf.int32, name='prediction')

        with tf.name_scope('loss'):
            losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.sparse_label_input, logits=self.logits)
            self.loss_sum = tf.reduce_sum(losses)
            self.loss = tf.reduce_mean(losses, name='loss')

        with tf.name_scope('accuracy'):
            correct_predction = tf.equal(self.prediction, self.sparse_label_input)
            self.correct_num = tf.reduce_sum(tf.cast(correct_predction, tf.float16), name='accuracy')
            self.accuracy = tf.reduce_mean(tf.cast(correct_predction, tf.float16), name='accuracy')
        
        self.learning_rate = 1e-3
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        gs_vs = optimizer.compute_gradients(self.loss)
        self.train_op = optimizer.apply_gradients(gs_vs)

## 开始训练

In [4]:
train_x, train_y = train_data['data'], [1 if i == 'pos' else 0 for i in train_data['label']]
test_x, test_y = test_data['data'], [1 if i == 'pos' else 0 for i in test_data['label']]
cnn_model = TextCNN_layer(seq_length=train_x.shape[1], class_num=2, ebd_weights=ebd_weights, filter_num=32)
batch_size = 1000
epoch_max = 50


config = use_gpu_polite()
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    epoch_now = sess.run(cnn_model.global_step)
    
    while epoch_now < epoch_max:
        # 在训练集上按batch训练完所有，算作一个epoch
        batch_num = train_x.shape[0] // batch_size
        for i in range(batch_num+1):
            s_i = i * batch_size
            e_i = min((i+1)*batch_size, train_x.shape[0])
            if s_i >= e_i:
                continue
            in_x, in_y = train_x[s_i: e_i, :], train_y[s_i: e_i]
            feed_dict = {cnn_model.seq_input: in_x, cnn_model.sparse_label_input: in_y}
            sess.run(cnn_model.train_op, feed_dict)

        epoch_now = sess.run(cnn_model.global_step_op)  # 跑完了一个epoch，epoch+1
        
        if epoch_now % 10 == 0:  # 每10轮观察一下训练集测试集loss 和 acc
            # 训练集总的损失和acc也要分步测，否则内存不够
            batch_num = train_x.shape[0] // batch_size
            train_total_loss = 0
            train_total_correct = 0
            for i in range(batch_num+1):
                s_i = i * batch_size
                e_i = min((i+1)*batch_size, train_x.shape[0])
                if s_i >= e_i:
                    continue
                in_x, in_y = train_x[s_i: e_i, :], train_y[s_i: e_i]
                feed_dict = {cnn_model.seq_input: in_x, cnn_model.sparse_label_input: in_y}

                train_loss_one, train_correct_one = sess.run([cnn_model.loss_sum, cnn_model.correct_num], feed_dict)
                train_total_loss += train_loss_one
                train_total_correct += train_correct_one
            train_loss = train_total_loss / train_x.shape[0]
            train_acc = train_total_correct / train_x.shape[0]

            # 测试集的损失和acc
            batch_num = test_x.shape[0] // batch_size
            test_total_loss = 0
            test_total_correct = 0
            for i in range(batch_num+1):
                s_i = i * batch_size
                e_i = min((i+1)*batch_size, test_x.shape[0])
                if s_i >= e_i:
                    continue
                in_x, in_y = test_x[s_i: e_i, :], test_y[s_i: e_i]
                feed_dict = {cnn_model.seq_input: in_x, cnn_model.sparse_label_input: in_y}

                test_loss_one, test_correct_one = sess.run([cnn_model.loss_sum, cnn_model.correct_num], feed_dict)
                test_total_loss += test_loss_one
                test_total_correct += test_correct_one
            test_loss = test_total_loss / test_x.shape[0]
            test_acc = test_total_correct / test_x.shape[0]

            flatten_param, logits = sess.run([cnn_model.pool_flatten, cnn_model.logits], 
                                             {cnn_model.seq_input: train_x[0:1, :]})
            print('Epoch %d, train loss %.4f, acc %.4f; test loss %.4f, acc %.4f' % 
                  (epoch_now, train_loss, train_acc, test_loss, test_acc))
            print(flatten_param, '\n', logits)


Epoch 10, train loss 4.9552, acc 0.6108; test loss 5.5408, acc 0.5916
[[ 7.1507683  0.9235705  8.771172   4.0647635  4.441228   4.078831
   4.686659   4.563436   5.448373   3.3299072  3.1972177  5.536707
   4.7202597  4.5665407  3.3574715  3.9295497  5.0851145  5.815064
   5.3960667  6.717537   4.6561666  3.429185   5.158939   4.4089956
   2.8463964  3.0861168  2.144047   4.4400115  6.0825334  4.400116
   4.140744   5.149525   5.54589    8.144511  11.44268    8.524857
   7.023779   5.759207   4.148216   6.2796974  5.3028364  4.8843536
   4.455388   6.9233866  6.0895743  7.3255033  3.6607678  3.5709817
   4.6960087  5.728938   6.2494655  6.6449275  5.6996465  1.3828605
   4.6555014  3.9797611  3.461446   4.642683   2.8416858  6.8132925
   6.197632   5.5107903  3.4199927  3.0365307  6.5887294  7.9431195
   7.456849   6.758239   6.1292887  5.710578   6.460667   6.791635
   6.988167   6.355074   4.7688093  6.770875   9.24423    9.064208
   3.5329266  5.6233883  5.7817974  6.2300353  9.2213

**两个4-开头的文件都是实现TextCNN，这个使用的是lyaers接口，另一个扎扎实实地写conv和dense的每个参数。**  
**两种方法使用的参数初始化方法完全一样，随机种子也一样，都设置成7。期待训练结果在loss和acc的数值上完全一致。**  
**但是，还是存在细微差异，虽然总体上差别不大（几乎小数点后三位）。**  
**同时还打出了pooling之后的tensor，以及logits。发现pooling之后就有细微差距了，应该就是conv层产生了不同。**  
**猜测一：可能是因为conv有32个kernel，那里随机初始化两种方法有偏差。**  
**猜测二：还存在其它参数初始化存在随机，或者训练过程中有随机。因为两次运行同一个文件，flatten层输出也不一样。**  
**可以看到，layers接口集成度高，方便很多。事实上，可能底层方法和就和自己写的差不多。所以之后尽量都用layers接口去实现模型。**